In [ ]:
import grpc
from proto import account_pb2, account_pb2_grpc, events_pb2, events_pb2_grpc
from google.protobuf import empty_pb2
import pytz
from datetime import datetime, timedelta
from couchers.utils import Timestamp_from_datetime

utc = pytz.UTC

In [ ]:
class CookieCreds:
    def __init__(self, cookie_name, cookie_value):
        self.cookie_name = cookie_name
        self.cookie_value = cookie_value
        
    def __call__(self, context, callback):
        callback((("cookie", f"{self.cookie_name}={self.cookie_value}"),), None)

In [ ]:
def create_channel(api_key, server_address="api.couchers.org:8443"):
    channel_creds = grpc.ssl_channel_credentials()
    cookie_creds = grpc.metadata_call_credentials(CookieCreds("couchers-sesh", api_key))
    creds = grpc.composite_channel_credentials(channel_creds, cookie_creds)
    return grpc.secure_channel(server_address, creds)

In [ ]:
# paste cookie here:
api_key = "e22...a4f"

In [ ]:
channel = create_channel(api_key, "dev-api.couchershq.org:8443")

In [ ]:
events = events_pb2_grpc.EventsStub(channel)

In [ ]:
start = utc.localize(datetime(2021, 7, 4, 19, 0, 0))
end = start + timedelta(hours=1)

res = events.CreateEvent(events_pb2.CreateEventReq(
  title="Weekly Online Event (Tuesday)",
  content="Hello! We are running this online weekly event tailored to fit European, Asian, and Oceanian time zones. As usual, please respond to let us know if there is a better time that would suit you, as this time slot is flexible :). Or, you maybe be able to attend [our other weekly event time](TODO).",
  online_information=events_pb2.OnlineEventInformation(link="https://couchers.org/weekly-social"),
  parent_community_id=1,
  start_time=Timestamp_from_datetime(start),
  end_time=Timestamp_from_datetime(end),
  timezone="UTC",
))

events.TransferEvent(
    events_pb2.TransferEventReq(
        event_id=res.event_id,
        new_owner_community_id=1,
    )
)